In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("주식분석")

LangSmith 추적을 시작합니다.
[프로젝트명]
주식분석


In [3]:
# 필요 라이브러리 임포트
import os
import json
from pydantic import BaseModel, Field
from typing import Annotated, List, Dict
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool

In [4]:
######## states.py ########
class OverallState(TypedDict):
    user_input: str
    messages: Annotated[list, add_messages]
    Stock_Value_dict: dict

class InputState(TypedDict):
    start_input: str
    
class StockValueState(TypedDict):
    user_input: str
    messages: Annotated[list, add_messages]
    Stock_Value_dict: dict
    retrieve_check: bool
    retrieval_msg: str
    rewrite_query: str
    tools_call_switch: Annotated[bool, True]

class SearchQueryState(TypedDict):
    messages: Annotated[list, add_messages]
    Stock_Value_dict: dict
    query_list: list
    previous_query: list
    is_revise: bool
    
class EndState(TypedDict):
    messages: Annotated[list, add_messages]
    query_list: list

In [5]:
######## nodes.py ########
# ChromaDB 로드
vectorstore = Chroma(
    collection_name="rag-chroma",
    embedding_function=OpenAIEmbeddings(),
    persist_directory="./chroma_db",
)
retriever = vectorstore.as_retriever()

In [6]:
# 시작노드 - 페르소나에 대한 정보를 요구하는 노드임
def user_input_node(state: InputState):
    print("================================= calculation stock =================================")
    print("주식 가치를 분석합니다. 궁금하신 주식명을 말씀해주세요.")
    # time.sleep(1)
    user_input = input("User: ")
    
    return {"messages": [("user", user_input)], "tools_call_switch": True}

# ----------------------------------------------------------------------------
# ----------------------------------------------------------------------------
# ----------------------------------------------------------------------------
# 노드 1 - 입력된 문장으로부터 필요한 정보를 만들어내는 노드.
# 검색용 Tavily 툴 로드하고 노드만듦.
tool = TavilySearchResults(max_results=3)
web_search_tool = TavilySearchResults(max_results=5)

# 노드 1-1. 검색용 노드
tool_node = ToolNode(tools=[tool])

# 검색용 RAG 툴 로드하고 노드만듬
retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_report",
    """
    Search and return the following financial information for the company specified by User Input:
    Net income
    Shares outstanding
    Current stock price
    Shareholder's equity
    Free cash flow
    Operating income
    Weighted Average Cost of Capital (WACC)
    Projected future cash flows
    Growth rate
    Dividend per share
    Other return-related information (e.g., ROI, ROE, ROA)"
    """
)
# 노드 1-2. RAG용 노드.
retrieve = ToolNode([retriever_tool])

def tool_nodes_exporter():
    return tool_node, retrieve

# 두 개 툴 엮어서 리스트 만듦.
tools = [tool, retriever_tool]

In [7]:
# 노드 1-3. RAG 검증노드
# 노드 1-2의 Tools Output을 받아서, User Input에 잘 맞는지 검증해서 Yes Or No로 대답함.
# 만약 Yes라면 그대로 다시 Character Make Node로 보내서 최종 답변을 생성하도록 하고
# 아니라면 검색을 진행하고 새로운 값을 받아서 보낼거임.

class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    binary_score:str = Field(..., description="Documents are relevant to the question, 'yes' or 'no'", enum=['yes', 'no'])

rag_check_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
rag_check_model = rag_check_model.with_structured_output(GradeDocuments)

def retrieve_check_node(state: StockValueState):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """
            You are a financial professional who provides appropriate information in response to user input.
            Return 'yes' or 'no' if you can provide an accurate answer to the user's question from the given documentation.
            If you can't provide a clear answer, be sure to return NO.
            """),
            ("human", "Retrieved document: \n\n {document} \n\n User's input: {question}"),
        ]
    )
    
    retrieval_msg = state['messages'][-1].content
    human_msg = state['user_input']
    retrieval_grader = prompt | rag_check_model
    response = retrieval_grader.invoke({"document": retrieval_msg, "question": human_msg})
    retrieve_handle = response.binary_score
    retrieve_check = False
    
    if retrieve_handle == "no":
        print("=============================== Need to Check ===============================")
        retrieve_check = True
    if retrieve_handle == "yes":
        print("============================== No Need to Check =============================")
        
    return {"retrieve_check": retrieve_check, "retrieval_msg": retrieval_msg}

In [8]:
# 노드 1-4. 쿼리 재-작성 노드
# 노드 1-2에서 산출된 retrieve가 입력값과 적절하게 매치되지 않는 경우, 입력값을 수정하게 됨.
# state User_input 이용
# 이는 노드 1-3에서 yes를 반환하는 경우에 실행됨.

class Rewrite_Output(TypedDict):
    """
    Sturctured_output을 생성하기위한 클래스
    """
    query: Annotated[str, ..., "Rewritten query to find appropriate material on the web"]

rewrite_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
rewrite_model = rewrite_model.with_structured_output(Rewrite_Output)

def rewrite_node(state: StockValueState):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """
            You're an expert in improving search relevance.\n
            Look at previously entered search queries and rewrite them to better find that information on the internet.
            """),
            ("human", "Previously entered search queries: \n{user_input}"),
        ]
    )
    
    user_input = state['user_input']
    rewrite_chain = prompt | rewrite_model
    response = rewrite_chain.invoke({"user_input": user_input})
    rewrited_query = response['query']
    print(f"================================ Rewrited Query ================================\nRewritted Query: {rewrited_query}")

    return {"rewrite_query": rewrited_query}

In [9]:
# 노드 1-5. 재작성된 쿼리를 이용해서 인터넷 검색하는 노드

def rewrite_search_node(state: StockValueState):
    print("================================ Search Web ================================")
    docs = web_search_tool.invoke({"query": state['rewrite_query']})
    web_results = "\n\n".join([d["content"] for d in docs])
    web_results = web_results + "\n\n" + state['retrieval_msg']
    # print(web_results)

    new_messages = [ToolMessage(content=web_results, tool_call_id="tavily_search_results_json")]
            
    return {"messages": new_messages}


In [10]:
# 노드 1번 작성된 것.
# 인간 입력이랑 Retrieve를 받을 수 있는 놈임.

stock_value_model = ChatOpenAI(model="gpt-4o", temperature=0.2)
stock_value_model_with_tools = stock_value_model.bind_tools(tools)

def stock_value_node(state: StockValueState):
    prompt = ChatPromptTemplate.from_messages([
        ("system","""
        You are a financial expert who specializes in stock valuation.
        Based on the stock name entered by the user, you will calculate its fair value.

        To calculate the fair value, you will need the following information:
        '''
        - Net income
        - Shares outstanding
        - Current stock price
        - Shareholder's equity
        - Free cash flow
        - Operating income
        - WACC
        - Projected future cash flows
        - Growth rate
        - Dividend per share
        '''
        The final result should be presented in Korean.
        """),
        ("human", "Input: {human_input}\n Retrieve: {context}"),
    ])
    prompt_with_tools = ChatPromptTemplate.from_messages([
        ("system","""
        You are a financial expert who specializes in stock valuation.
        Based on the stock name entered by the user, you will calculate its fair value.

        To calculate the fair value, you will need the following information:
        '''
        - Net income
        - Shares outstanding
        - Current stock price
        - Shareholder's equity
        - Free cash flow
        - Operating income
        - WACC
        - Projected future cash flows
        - Growth rate
        - Dividend per share
        '''
        Solve the problem by searching online for the value of the information needed for fair value.
        
        The final result should be presented in Korean.
        """),
        ("human", "Input: {human_input}\n Retrieve: {context}"),
    ])
    messages_list = state['messages']
    last_human_message = next((msg for msg in reversed(messages_list) if isinstance(msg, HumanMessage)), None).content
    last_msg = state['messages'][-1].content
    
    if last_human_message == last_msg:
        last_msg = ""
        print(f"==================================== INPUT ====================================\nHuman Input: {last_human_message}")
    else:
        try:
            last_msg_data = json.loads(state['messages'][-1].content)
            last_msg = "\n\n".join([d["content"] for d in last_msg_data])
        except:
            ...
        print(f"==================================== INPUT ====================================\nHuman Input: {last_human_message}\nContext: {last_msg}")
    
    if state['tools_call_switch']:
        chain_with_tools = prompt_with_tools | stock_value_model_with_tools
        response = chain_with_tools.invoke({"human_input": last_human_message, "context": last_msg})
        
        if hasattr(response, "tool_calls") and len(response.tool_calls) > 0 and (response.tool_calls[0]["name"]) == "tavily_search_results_json":
            print("================================ Search Online ================================")
            tool_switch = False
        elif hasattr(response, "tool_calls") and len(response.tool_calls) > 0 and (response.tool_calls[0]["name"]) == "retrieve_report":
            print("=============================== Search Retrieval ===============================")
            tool_switch = False
        else:
            print("============================= Stock Value Information =============================")
            tool_switch = False
            print(response.content)
            
    else:
        chain = prompt | stock_value_model
        response = chain.invoke({"human_input": last_human_message, "context": last_msg})
        print("============================= Stock Value Information =============================")
        tool_switch = False
        print(response.content)

    return {"messages": [response], "user_input": last_human_message, "tools_call_switch": tool_switch}

In [11]:
# 노드2 - 입력된 문장으로부터 페르소나에 관한 정보를 추출하고, 정보가 없는 경우 이를 채워넣는 노드.
class Stock_value_output(TypedDict):
    """
    Sturctured_output을 생성하기위한 클래스
    """
    Net_Income: Annotated[str, ..., "순이익"]
    Shares_Outstanding: Annotated[str, ..., "발행주식수"]
    Stock_Price: Annotated[str, ..., "주가"]
    Shareholders_equity: Annotated[str, ..., "자기자본"]
    Free_cash_flow: Annotated[str, ..., "자유현금흐름"]
    Operating_income: Annotated[str, ..., "영업이익"]
    WACC: Annotated[str, ..., "할인율"]
    Projected_future_cash_flows: Annotated[str, ..., "미래 현금흐름"]
    Growth_Rate: Annotated[str, ..., "성장률"]
    Dividend_per_share: Annotated[str, ..., "주당배당금"]
    
stock_value_cal_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
stock_value_cal_model = stock_value_cal_model.with_structured_output(Stock_value_output)

# 페르소나를 반환하는 매우 경직된 LLM.
# 정보가 없는 경우 임의의 값을 채워넣도록 되어있음.
import pprint

def value_calculation_node(state: StockValueState):
    messages = [
        ("system", """
        You are a financial expert who specializes in stock valuation.
        Based on the stock name entered by the user, you will calculate its fair value.

        To calculate the fair value, you will need the following information:
        '''
        - Net income
        - Shares outstanding
        - Current stock price
        - Shareholder's equity
        - Free cash flow
        - Operating income
        - WACC
        - Projected future cash flows
        - Growth rate
        - Dividend per share
        '''
        The final result should be presented in Korean.
        """),
        ("human", state['messages'][-1].content)
    ]
    response = stock_value_cal_model.invoke(messages)

    print("================================= stock value calculation Setup =================================")
    print(f"입력된 정보:{state['user_input']}")
    print(f"순이익: {response['Net_Income']}")
    print(f"발행주식수: {response['Shares_Outstanding']}")
    print(f"주가: {response['Stock_Price']}")
    print(f"자기자본: {response['Shareholders_equity']}")
    print(f"자유현금흐름: {response['Free_cash_flow']}")
    print(f"영업이익: {response['Operating_income']}")
    print(f"할인율: {response['WACC']}")
    print(f"미래 현금흐름: {response['Projected_future_cash_flows']}")
    print(f"성장률: {response['Growth_Rate']}")
    print(f"주당배당금: {response['Dividend_per_share']}")

    pprint.pprint(response)
    
    return {"Stock_Value_dict": response}


In [12]:
# 노드 3 - 페르소나를 토대로 적절한 검색 키워드를 생성하는 놈.

class Search_Output(TypedDict):
    """
    Sturctured_output을 생성하기위한 클래스
    """
    query_list: Annotated[list, ..., "List of queries that customers have entered in your shop"]

search_model = ChatOpenAI(model="gpt-4o-mini")
search_model = search_model.with_structured_output(Search_Output)

examples = [
    {"input": 
        """
        stock name: 삼성전자,
        stock Net income: 36.6조 원,
        stock Shares Outstanding: 5.5억 주,
        stock Current stock price: 64,000원,
        stock Shareholders equity: 300조 원,
        stock Free cash flow: 30조 원,
        stock Operating income: 45조 원,
        stock WACC: 7.5%,
        stock Projected future cash flows: 25배,
        stock Growth Rate: 10%,
        stock Dividend per share: 1,152 원
        """, 
    "output": 
        ['PBR : 2배', 'PER : 12배', 'DCF : 70,000 원', 'DDM : 54,000 원', 'stock value : 63,000 원']
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

def search_setence_node(state: SearchQueryState):
    prompt = ChatPromptTemplate.from_messages([
        ("system","""
        You're a great financial analyst, and you're working on inferring stock values.
        Given information, infer the appropriate stock value.
        """),
        few_shot_prompt,
        ("human", """
        stock Net income: {Net_Income},
        stock Shares Outstanding: {Shares_Outstanding},
        stock Current stock price: {Stock_Price},
        stock Shareholders equity: {Shareholders_equity},
        stock Free cash flow: {Free_cash_flow},
        stock Operating income: {Operating_income},
        stock WACC: {WACC},
        stock Projected future cash flows: {Projected_future_cash_flows},
        stock Growth Rate: {Growth_Rate},
        stock Dividend per share: {Dividend_per_share},
         """),
    ])
    
    chain = prompt | search_model
    response = chain.invoke(
        {
            "Net_Income": state['Stock_Value_dict']['Net_Income'],
            "Shares_Outstanding": state['Stock_Value_dict']['Shares_Outstanding'],
            "Stock_Price": state['Stock_Value_dict']['Stock_Price'],
            "Shareholders_equity": state['Stock_Value_dict']['Shareholders_equity'],
            "Free_cash_flow": state['Stock_Value_dict']['Free_cash_flow'],
            "Operating_income": state['Stock_Value_dict']['Operating_income'],
            "WACC": state['Stock_Value_dict']['WACC'],
            "Projected_future_cash_flows": state['Stock_Value_dict']['Projected_future_cash_flows'],
            "Growth_Rate": state['Stock_Value_dict']['Growth_Rate'],
            "Dividend_per_share": state['Stock_Value_dict']['Dividend_per_share'],
        }
    )
    print("=============================== Search Queries ===============================")
    print(response['query_list'])
    
    return {"query_list": response}

In [13]:
# 노드 4, revise_tool - 반환된 서치쿼리가 적당한지 검증하는 노드임.
class QueryReviseAssistance(BaseModel):
    """Escalate the conversation. 
    Use only if the given search query is a strong mismatch with the customer's information.
    Use this tool even if given search query is seriously inappropriate to enter into the search bar of an online retailer like Amazon.
    Never call the tool if the same input is still being given as before.
    To use this function, return 'query_list'.
    """
    query_list: list
    
query_check_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5, streaming=True)
query_check_model = query_check_model.bind_tools([QueryReviseAssistance])

def query_check_node(state: SearchQueryState):
    print("=============================== Query Check ===============================")
    prompt = ChatPromptTemplate.from_messages([
        ("system","""
        You are a search manager.
        If you think that the given customer's information and the search query that they used on your online store are relevant, then return the query as it is.
        Never invoke the tool if you are still being given the same query that was entered in the previous dialogue.
        """),
        ("human", """
            stock Net income: {Net_Income},
            stock Shares Outstanding: {Shares_Outstanding},
            stock Current stock price: {Stock_Price},
            stock Shareholders equity: {Shareholders_equity},
            stock Free cash flow: {Free_cash_flow},
            stock Operating income: {Operating_income},
            stock WACC: {WACC},
            stock Projected future cash flows: {Projected_future_cash_flows},
            stock Growth Rate: {Growth_Rate},
            stock Dividend per share: {Dividend_per_share},
            Queries: {queries}
            """),
        ])
    chain = prompt | query_check_model
    
    response = chain.invoke(
        {
            "Net_Income": state['Stock_Value_dict']['Net_Income'],
            "Shares_Outstanding": state['Stock_Value_dict']['Shares_Outstanding'],
            "Stock_Price": state['Stock_Value_dict']['Stock_Price'],
            "Shareholders_equity": state['Stock_Value_dict']['Shareholders_equity'],
            "Free_cash_flow": state['Stock_Value_dict']['Free_cash_flow'],
            "Operating_income": state['Stock_Value_dict']['Operating_income'],
            "WACC": state['Stock_Value_dict']['WACC'],
            "Projected_future_cash_flows": state['Stock_Value_dict']['Projected_future_cash_flows'],
            "Growth_Rate": state['Stock_Value_dict']['Growth_Rate'],
            "Dividend_per_share": state['Stock_Value_dict']['Dividend_per_share'],
            "queries": state['query_list']['query_list'],
        }
    )
    is_revise = False
        
    if (
        response.tool_calls
        and response.tool_calls[0]["name"] == QueryReviseAssistance.__name__
    ):
        print("Revise Requires")
        is_revise = True
    
    return {"messages": [response], "is_revise": is_revise}


In [14]:
# 노드 4-1. 쿼리를 수정하도록 요청받은 경우 이를 수행하는 노드임.

class QueryCheck_Output(TypedDict):
    """
    Sturctured_output을 생성하기위한 클래스
    """
    query_list: Annotated[list, ..., "List of queries that customers might have entered in search-bar of your online retail shop"]
    
query_revise_model = ChatOpenAI(model="gpt-4o")
query_revise_model = query_revise_model.with_structured_output(QueryCheck_Output)

def query_revise_node(state: SearchQueryState):
    print("=============================== Query Revise ===============================")
    prompt = ChatPromptTemplate.from_messages([
        ("system",
            """
                You are a validator who fixes errors in a given query.
                From the list of queries given, remove or modify the queries that do not match the user's information appropriately.
                Be sure to delete highly irrelevant data.
                Be sure to remove search terms that you wouldn't use on a shopping site like Amazon.
                Return the modified queries as a list.
            """
        ),
        ("human", 
            """
                stock Net income: {Net_Income},
                stock Shares Outstanding: {Shares_Outstanding},
                stock Current stock price: {Stock_Price},
                stock Shareholders equity: {Shareholders_equity},
                stock Free cash flow: {Free_cash_flow},
                stock Operating income: {Operating_income},
                stock WACC: {WACC},
                stock Projected future cash flows: {Projected_future_cash_flows},
                stock Growth Rate: {Growth_Rate},
                stock Dividend per share: {Dividend_per_share}
            """
        )])
    
    chain = prompt | query_revise_model
    response = chain.invoke(
        {
            "Net_Income": state['Stock_Value_dict']['Net_Income'],
            "Shares_Outstanding": state['Stock_Value_dict']['Shares_Outstanding'],
            "Stock_Price": state['Stock_Value_dict']['Stock_Price'],
            "Shareholders_equity": state['Stock_Value_dict']['Shareholders_equity'],
            "Free_cash_flow": state['Stock_Value_dict']['Free_cash_flow'],
            "Operating_income": state['Stock_Value_dict']['Operating_income'],
            "WACC": state['Stock_Value_dict']['WACC'],
            "Projected_future_cash_flows": state['Stock_Value_dict']['Projected_future_cash_flows'],
            "Growth_Rate": state['Stock_Value_dict']['Growth_Rate'],
            "Dividend_per_share": state['Stock_Value_dict']['Dividend_per_share'],
            "queries": state['query_list']['query_list'],
        }
    )

    return {"query_list": response, "is_revise": False}

In [15]:
######## edges.py ########
# 라우팅을 위한 함수
def select_next_node(state: SearchQueryState):
    if state["is_revise"]:
        return "is_revise"
    
    return '__end__'

def simple_route(state: StockValueState):
    """
    Simplery Route Tools or Next or retrieve
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0 and ai_message.tool_calls[0]["name"] == "tavily_search_results_json":
        # print("Tavily Search Tool Call")
        return "tools"
    elif hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0 and ai_message.tool_calls[0]["name"] == "retrieve_report":
        # print("Retrieve Call")
        return "retrieve"

    return "next"

def retrieve_route(state: StockValueState):
    """
    RAG Need Check?
    """
    if state['retrieve_check']:
        return "rewrite"

    return "return"

tool_node, retrieve = tool_nodes_exporter()

In [16]:
# 추가적인 필요사항 정리하고 그래프 빌딩
memory = MemorySaver()
graph_builder = StateGraph(OverallState, input=InputState, output=EndState)

graph_builder.add_node("User Input", user_input_node)
graph_builder.add_node("Stock Value calculation", stock_value_node)
graph_builder.add_node("Stock Retrieve Check", retrieve_check_node)
graph_builder.add_node("Rewrite Tool", rewrite_node)
graph_builder.add_node("Rewrite-Search", rewrite_search_node)
graph_builder.add_node("Value Calculation", value_calculation_node)
graph_builder.add_node("Search Sentence", search_setence_node)
graph_builder.add_node("Query Check", query_check_node)
graph_builder.add_node("Query Revise Tool", query_revise_node)
graph_builder.add_node("Tavily Search Tool", tool_node)
graph_builder.add_node("RAG Tool", retrieve)

graph_builder.add_edge(START, "User Input")
graph_builder.add_edge("User Input", "Stock Value calculation")
graph_builder.add_edge("Tavily Search Tool", "Stock Value calculation")
graph_builder.add_edge("RAG Tool", "Stock Retrieve Check")
graph_builder.add_edge("Rewrite Tool", "Rewrite-Search")
graph_builder.add_edge("Rewrite-Search", "Stock Value calculation")
graph_builder.add_edge("Value Calculation", "Search Sentence")
graph_builder.add_edge("Search Sentence", "Query Check")
graph_builder.add_edge("Query Revise Tool", "Query Check")
graph_builder.add_conditional_edges(
    "Query Check", 
    select_next_node, 
    {"is_revise": "Query Revise Tool", END: END}
)
graph_builder.add_conditional_edges(
    "Stock Value calculation",
    simple_route,
    {"tools": "Tavily Search Tool", "next": "Value Calculation", "retrieve": "RAG Tool"}
)
graph_builder.add_conditional_edges(
    "Stock Retrieve Check", 
    retrieve_route, 
    {"rewrite": "Rewrite Tool", "return": "Stock Value calculation"}
)

In [17]:
##### edges.py에서 Graph Export #####
def Project_Graph():
    graph = graph_builder.compile(checkpointer=memory)
    return graph

In [18]:
####### run_graph.py #######

graph = Project_Graph()
config = {"configurable": {"thread_id": "1"}}

with open("graph_output.png", "wb") as f:
    f.write(graph.get_graph().draw_mermaid_png())
    
graph.invoke({"start_input": ""}, config=config)

================================= calculation stock =================================
주식 가치를 분석합니다. 궁금하신 주식명을 말씀해주세요.


User:  삼성전자


==================================== INPUT ====================================
Human Input: 삼성전자
=============================== Search Retrieval ===============================
=============================== Need to Check ===============================
================================ Rewrited Query ================================
Rewritted Query: 삼성전자 최신 뉴스 및 제품 정보
================================ Search Web ================================
==================================== INPUT ====================================
Human Input: 삼성전자
Context: 삼성전자 뉴스룸(Samsung Newsroom Korea) – 삼성전자 뉴스룸은 삼성전자의 대표 뉴스 채널입니다.
삼성전자 뉴스룸의 동영상 콘텐츠는 더 이상 Internet Explorer를 지원하지 않습니다. 최적의 시청 환경을 위해 다른 웹브라우저 사용을 권장합니다.
close
본문 바로가기
Samsung Newsroom Korea – 삼성전자 뉴스룸은 삼성전자의 대표 뉴스 채널입니다.


기업뉴스
기업문화
기술
디자인
경영일반


제품뉴스
모바일
TV/디스플레이
가전
반도체
더 많은 제품


미래동행
사회공헌
친환경
지속가능경영


프레스센터
보도자료
미디어 라이브러리
이슈와 팩트
회사소개(Fast-Facts)



메뉴 열기 검색 레이어 열기 삼성 뉴스룸 국가 선택
검색 닫기
검색  검색어 삭제    검색

갤럭시 S25
CES 2025
AI
One UI
CSR

상세검색


{'messages': [HumanMessage(content='삼성전자', additional_kwargs={}, response_metadata={}, id='c08b830a-2ee7-4913-97fc-a6c8987c21e9'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wCIYIUgpHp7TBBzzcAF0Lo4M', 'function': {'arguments': '{"query":"삼성전자"}', 'name': 'retrieve_report'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 331, 'total_tokens': 348, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4ecdfc8cdc', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-68bf8691-4177-42b8-ab65-bc72202c3f7a-0', tool_calls=[{'name': 'retrieve_report', 'args': {'query': '삼성전자'}, 'id': 'call_wCIYIUgpHp7TBBzzcAF0Lo4M', 'type': 'tool_call'}], usage_metadata={'input_tokens': 331, 'output_t